In [12]:
# Librerias
import pandas as pd
import unidecode

In [19]:
# Importamos los archivos
df_patente67 = pd.read_csv('INVENTOR6724.csv')
df_patenteMX = pd.read_excel('INVENTORES_MX.xlsx')

In [20]:
# Hacemos modificaciones en 'df_patente67'
# 'inventor_first_name' y 'inventor_last_name' se uniran en una sola columna llamada 
# 'NOMBRE_INVENTOR' en el siguiente formato inventor_last_name,inventor_first_name
df_patente67['NOMBRE_INVENTOR'] = df_patente67['inventor_last_name'] + ',' + df_patente67['inventor_first_name']

# Eliminamos las columnas que no necesitamos
df_patente67 = df_patente67.drop(columns=['inventor_first_name', 'inventor_last_name'])

# Eliminamos guiones, comas, numero y puntos en la columna 'NOMBRE_INVENTOR'
df_patente67['NOMBRE_INVENTOR'] = df_patente67['NOMBRE_INVENTOR'].str.replace('-', ' ').str.replace('[0-9]', '').str.replace('.', '').apply(unidecode.unidecode)

In [ ]:
# Hacemos modificaciones en 'df_patente67'
# 'inventor_city' y 'inventor_country' se uniran en una sola columna llamada
# 'Location' en el siguiente formato inventor_city,inventor_country
df_patente67['Location'] = df_patente67['inventor_city'] + ', ' + df_patente67['inventor_country']

# Eliminamos las columnas que no necesitamos
df_patente67 = df_patente67.drop(columns=['inventor_lastknown_city', 'inventor_lastknown_country','inventor_country', 'inventor_city'])

In [ ]:
# Eliminamos las columnas que no necesitamos en 'df_patente67'
df_patente67 = df_patente67.drop(columns=['inventor_key', 'inventor_id', 'patent_id', 'inventor_order', 'inventor_sequence'])